In [30]:
import os
import re
import json
import gzip
import time
import zipfile
import requests
import numpy as np
import pandas as pd
from datetime import date
from decimal import Decimal
CACHE = {} # глобальный кэш, в процессе скрейпинга не обновлять!
SETS = './sets/' # папка с файлами с наборами

In [31]:
def default_serializer(obj): # для правильного преобразования даты в ISO формат
    if isinstance(obj, (date)):
        return obj.isoformat()
    raise TypeError(f'Object of type {obj.__class__.__name__} is not JSON serializable')
    

def load_cache(filename='cache.gz'): # загрузка кэша
    global CACHE
    if os.path.exists(filename) and len(CACHE)==0:
        with gzip.open(filename, 'rb') as gzip_ref:
            CACHE = json.load(gzip_ref)
        print(f'Загружено {len(CACHE)} записей из {filename} в кэш')


def save_cache(filename='cache.gz'): # сохранение кэша
    global CACHE
    with gzip.open(filename, 'wb') as gzip_file:
        json_data = json.dumps(CACHE, ensure_ascii=False, default=default_serializer)
        gzip_file.write(json_data.encode('utf-8'))
    print(f'Сохранено {len(CACHE)} записей в {filename}')
    
    
def load_gzon(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return json.load(gzip_ref)
    else:
        return []
    

def save_gzon(filename):
    data = globals().get(filename)
    if data is not None:
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(json_data)} записей в {filename}.gz')
    else:
        print(f'Переменная {filename} не найдена')
        
     
def load_dataset(filename): 
    if os.path.exists(SETS + filename + '.gz'):
        with gzip.open(SETS + filename + '.gz', 'rb') as gzip_ref:
            return pd.DataFrame(json.load(gzip_ref))
        
    
def save_dataset(filename):
    data = globals().get(filename)
    if data is not None:
        data = data.to_dict(orient='records')
        json_data = json.dumps(data, ensure_ascii=False, default=default_serializer)
        with gzip.open(SETS + filename + '.gz', 'wb') as gzip_file:
            gzip_file.write(json_data.encode('utf-8'))
        print(f'Сохранено {len(data)} записей в {filename}.gz')

In [53]:
pd.set_option('display.max_colwidth', None)
ecology = load_dataset('ecology')

In [56]:
ecology[ecology['start'].str.len() < 7]

,start,end,air,mean,noise,dirt,digging,capital,house_lines,clinics,...,city_places,live_area,streets,trashes,factories,markets,autostations,gasstations,bad_gasstations,buildgrounds
12100,,,NaN,NaN,NaN,NaN,NaN,"[{'boundingbox': ['55.8234233', '55.8235233', '37.8111564', '37.8112564']}]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12101,,,NaN,NaN,NaN,NaN,NaN,"[{'boundingbox': ['55.8263043', '55.8265621', '37.8122507', '37.8125184']}]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12102,,,NaN,NaN,NaN,NaN,NaN,"[{'boundingbox': ['55.8270303', '55.8274414', '37.8121345', '37.8127507']}]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13990,,,NaN,NaN,NaN,NaN,NaN,"[{'boundingbox': ['55.7946236', '55.7948557', '37.7158353', '37.7171764']}]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13991,,,NaN,NaN,NaN,NaN,NaN,"[{'boundingbox': ['55.8005394', '55.8006101', '37.7448243', '37.7449765']}]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185821,,26.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.400555311, 55.643330548], 'type': 'Point'}"
185822,,26.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.65791113, 55.637167012], 'type': 'Point'}"
185823,,26.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.735652297, 55.776123852], 'type': 'Point'}"
185824,,26.07.2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'coordinates': [37.564242096, 55.663143468], 'type': 'Point'}"


In [45]:
ecology['start'] = pd.to_datetime(ecology['start'],format='%d.%m.%Y', errors='coerce')
ecology['end'] = pd.to_datetime(ecology['end'], format='%d.%m.%Y', errors='coerce')

In [57]:
capital = load_dataset('62963')
capital.head()

,ObjectAddress,AdmArea,District,WorkEssence,DetailedWork,WorkName,YearOfWork,WorkVolume,ContractType,ContractNumber,...,ContractorName,ContractStartDate,ContractEndDate,ContractStatus,StartDate,StartDateActual,EndDate,EndDateActual,WorkStatus,global_id
0,"[{'is_deleted': 0, 'global_id': 13495, 'Address': 'Российская Федерация, город Москва, внутригородская территория муниципальный округ Лосиноостровский, Стартовая улица, дом 11', 'UNOM': '24733'}]",Северо-Восточный административный округ,Лосиноостровский район,Розлив системы холодного водоснабжения,Ремонт,Ремонт Розлив системы холодного водоснабжения,2014,"[{'is_deleted': 0, 'global_id': 79997, 'Volume': 594, 'MeasuringUnit': 'погонный метр'}]",государственный контракт,0373200017514000008,...,"Общество с ограниченной ответственностью ""КАФА""",03.02.2014,31.12.2014,Исполнен,15.01.2014,12.02.2014,25.08.2014,03.12.2014,завершена,1136596954
1,"[{'is_deleted': 0, 'global_id': 7275, 'Address': 'Российская Федерация, город Москва, внутригородская территория муниципальный округ Лосиноостровский, Стартовая улица, дом 37', 'UNOM': '24747'}]",Северо-Восточный административный округ,Лосиноостровский район,Розлив системы горячего водоснабжения,Ремонт,Ремонт Розлив системы горячего водоснабжения,2014,"[{'is_deleted': 0, 'global_id': 72325, 'Volume': 394, 'MeasuringUnit': 'погонный метр'}]",государственный контракт,0373200017514000008,...,"Общество с ограниченной ответственностью ""КАФА""",03.02.2014,31.12.2014,Исполнен,15.01.2014,02.04.2014,25.08.2014,04.08.2014,завершена,1136594500
2,"[{'is_deleted': 0, 'global_id': 8054, 'Address': 'Российская Федерация, город Москва, внутригородская территория муниципальный округ Лосиноостровский, Изумрудная улица, дом 52', 'UNOM': '8765'}]",Северо-Восточный административный округ,Лосиноостровский район,Розлив системы горячего водоснабжения,Ремонт,Ремонт Розлив системы горячего водоснабжения,2014,"[{'is_deleted': 0, 'global_id': 73104, 'Volume': 170, 'MeasuringUnit': 'погонный метр'}]",государственный контракт,0373200017514000008,...,"Общество с ограниченной ответственностью ""КАФА""",03.02.2014,31.12.2014,Исполнен,15.01.2014,02.04.2014,25.08.2014,25.04.2014,завершена,1136595032
3,"[{'is_deleted': 0, 'global_id': 13611, 'Address': 'Российская Федерация, город Москва, внутригородская территория муниципальный округ Лосиноостровский, улица Малыгина, дом 1', 'UNOM': '13749'}]",Северо-Восточный административный округ,Лосиноостровский район,Розлив системы холодного водоснабжения,Ремонт,Ремонт Розлив системы холодного водоснабжения,2014,"[{'is_deleted': 0, 'global_id': 80113, 'Volume': 101, 'MeasuringUnit': 'погонный метр'}]",государственный контракт,0373200017514000008,...,"Общество с ограниченной ответственностью ""КАФА""",03.02.2014,31.12.2014,Исполнен,15.01.2014,05.03.2014,25.08.2014,04.08.2014,завершена,1136597430
4,"[{'is_deleted': 0, 'global_id': 13571, 'Address': 'Российская Федерация, город Москва, внутригородская территория муниципальный округ Марьина Роща, 3-я улица Марьиной Рощи, дом 22/28', 'UNOM': '13897'}]",Северо-Восточный административный округ,район Марьина Роща,Трубопровод системы канализации,Ремонт,Ремонт Трубопровод системы канализации,2014,"[{'is_deleted': 0, 'global_id': 80073, 'Volume': 80, 'MeasuringUnit': 'погонный метр'}]",государственный контракт,0373200017514000032,...,"Общество с ограниченной ответственностью ""КАФА""",24.02.2014,31.12.2014,Исполнен,15.01.2014,04.03.2014,25.08.2014,16.12.2014,завершена,1136596085
